In [0]:
bronzepath = "abfss://bronze@vehicletheft1.dfs.core.windows.net/"
# silverpath = "abfss://silver@vehicletheft1.dfs.core.windows.net/"
# goldpath = "abfss://gold@vehicletheft1.dfs.core.windows.net/"


In [0]:
df_location = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(bronzepath + "/bronze/locations.csv")  # Verify that 'bronzepath' is correctly defined and the file exists at this location.

display(df_location)

In [0]:
df_make_details = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(bronzepath + "/bronze/make_details.csv")  # Verify that 'bronzepath' is correctly defined and the file exists at this location.

display(df_make_details)

In [0]:
df_stolen_vehicles = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(bronzepath + "/bronze/stolen_vehicles.csv")  # Verify that 'bronzepath' is correctly defined and the file exists at this location.

display(df_stolen_vehicles)

In [0]:
df_stolen_vehicles_db_data_dictionary = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(bronzepath + "/bronze/stolen_vehicles_db_data_dictionary.csv")  # Verify that 'bronzepath' is correctly defined and the file exists at this location.

display(df_stolen_vehicles_db_data_dictionary)

In [0]:
# dbutils.fs.mount(
#     source = "wasbs://bronze@vehicletheft1.blob.core.windows.net/",
#     mount_point = "/mnt/bronze",
#     extra_configs = {
#         "fs.azure.account.key.vehicletheft1.blob.core.windows.net": dbutils.secrets.get('dbscopr', 'vehicltheft')
#     }
# )


In [0]:
df_location.columns

In [0]:
from pyspark.sql.functions import *


df_location= df_location.withColumn("population", regexp_replace(col("population"), ',', '').cast("int"))
display(df_location)

In [0]:
df_location = df_location.toDF(*[Column.lower().replace(' ', '_')   for Column in df_location.columns])
display(df_location)

In [0]:
df_stolen_vehicles = df_stolen_vehicles.toDF(*[Columns.lower().replace('_', ' ')   for Columns in df_stolen_vehicles.columns])
display(df_stolen_vehicles)



In [0]:
df_stolen_vehicles = df_stolen_vehicles.toDF(*[Columns.lower().replace(' ', '_')   for Columns in df_stolen_vehicles.columns])

display(df_stolen_vehicles)

In [0]:
display(df_stolen_vehicles_db_data_dictionary)
display(df_location)
display(df_make_details)
display(df_stolen_vehicles)

In [0]:
df_stolen_vehicles = df_stolen_vehicles.withColumnRenamed("vehicle_id", "vvehicle_id")
df_stolen_vehicles.show(2)

In [0]:
df_location.write.format("parquet").option("header", "true").save(bronzepath + "/silver/location")

In [0]:
df_make_details.write.format("parquet").option("header", "true").save(bronzepath+"/silver/makeDetails")

In [0]:
df_stolen_vehicles.write.format("parquet").option("header","true").save(bronzepath+"/silver/stolenVehicles")

In [0]:
df_stolen_vehicles_db_data_dictionary.write.format("parquet").option("header", "true").save(bronzepath+"/silver/stolenVehiclesDictionary")

In [0]:
df_location.show(1)

In [0]:
null_count_location = df_location.select([sum(when(col(column).isNull(), 1).otherwise(0)).alias(column) for column in df_location.columns])

null_count_stolenvehicle = df_stolen_vehicles.select([sum(when(col(column).isNull(), 1).otherwise(0)).alias(column) for column in df_stolen_vehicles.columns]) 

# null_count_data_dictornary = df_stolen_vehicles_db_data_dictionary.select([sum(when(col(column).isnull(), 0).otherwise(0)).alias("dictonaryCount") for column in df_stolen_vehicles_db_data_dictionary.columns])

null_count_make_details  =df_make_details.select([sum(when(col(column).isNull(), 1).otherwise(0)).alias(column)   for column in df_make_details.columns])

null_count_location.show()
null_count_stolenvehicle.show()
# null_count_data_dictornary.show()
null_count_make_details.show()


In [0]:
df_stolen_vehicles.printSchema()
null_count_stolenvehicle.show()

In [0]:
df_stolen_vehicles = df_stolen_vehicles.fillna({
    "vehicle_type":"No Data",
    "make_id":0,
    "model_year":0,
    "vehicle_desc":"No Data",
    "color":"No Data"
})

In [0]:
null_count_stolenvehicle = df_stolen_vehicles.select([sum(when(col(column).isNull(), 1).otherwise(0)).alias(column) for column in df_stolen_vehicles.columns]) 

null_count_stolenvehicle.show()

In [0]:
df_location.write.mode("overwrite").format("csv").save(bronzepath+"/gold/locations.csv")
df_make_details.write.mode("overwrite").format("csv").save(bronzepath+"/gold/makeDetails.csv") 
df_stolen_vehicles.write.mode("overwrite").format("csv").save(bronzepath+"/gold/stolenVehicles.csv")
df_stolen_vehicles_db_data_dictionary.write.mode("overwrite").format("csv").save(bronzepath+"/gold/stolenVehiclesDictionary.csv")

In [0]:
df_location.createOrReplaceTempView("location")
df_make_details.createOrReplaceTempView("makeDetails")  
df_stolen_vehicles.createOrReplaceTempView("stolenVehicles")
df_stolen_vehicles_db_data_dictionary.createOrReplaceTempView("stolenVehiclesDictionary")

In [0]:
%sql
select * from location

In [0]:
%sql
select * from stolenVehicles

In [0]:
%sql
select sv.model_year, l.region,l.location_id , count(vvehicle_id) num_of_vehicles_stolen  from stolenVehicles  sv inner join location l on sv.location_id = l.location_id
 group by sv.model_year, l.region ,l.location_id order by num_of_vehicles_stolen desc